<a href="https://colab.research.google.com/github/Rahul711sharma/Classification-ML/blob/main/Spam_mails_Naive_Bayes_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <u><b> Objective </b></u>
## <b> Your task is to predict whether a message will be spam or not. In the class we used the <code>sklearn.countvectorizer</code> to find vectors for each message. Now you need to do the same task but rather than using countvectorizer, you are required to use TF-IDF vectorizer to find the vectors for the messages. </b>

### You will use <code>tfidfVectorizer</code>. It will convert collection of text documents (SMS corpus) into 2D matrix. One dimension represent documents and other dimension repesents each unique word in SMS corpus.

### If $n^{th}$ term $t$ has occured $p$ times in $m^{th}$ document, $(m, n)$ value in this matrix will be $\rm TF-IDF(t)$, where 
$\rm TF-IDF(t) = \rm Term ~Frequency (TF) * \rm Inverse~ Document ~Frequency (IDF)$
* ### <b>Term Frequency (TF)</b> is a measure of how frequent a term occurs in a document.

* ### $TF(t)$= Number of times term $t$ appears in document ($p$) / Total number of terms in that document

* ### <b>Inverse Document Frequency (IDF)</b> is measure of how important term is. For TF, all terms are equally treated. But, in IDF, for words that occur frequently like 'is' 'the' 'of' are assigned less weight. While terms that occur rarely that can easily help identify class of input features will be weighted high.

* ###  $IDF(t)= log(\frac{\rm Total ~ number ~ of ~document}{ Number ~of~ documents~ with ~term ~t ~in~ it})$

### At end we will have for every message, vectors normalized to unit length equal to size of vocabulary (number of unique terms from entire SMS corpus)





In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
df= pd.read_csv("/content/drive/MyDrive/spam.csv",encoding='latin-1')

In [9]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [10]:
df= df.iloc[:,[0,1]]
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [16]:
df.columns = ['label','message']

In [17]:
df.groupby('label').describe()

message                                                               
        count unique                                                top freq
label                                                                       
ham      4825   4516                             Sorry, I'll call later   30
spam      747    653  Please call our customer service representativ...    4

In [18]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
import string
from nltk.corpus import stopwords

In [23]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [24]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [25]:
def text_process(msg):
    nopunc =[char for char in msg if char not in string.punctuation]
    nopunc=''.join(nopunc)
    return ' '.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english')])

In [26]:
text_process(df['message'])

'Go jurong point, crazy.. Available bugis n great world la e buffet... Cine got amore wat...Ok lar... Joking wif u oni...Free entry 2 wkly comp win FA Cup final tkts 21st May 2005. Text FA 87121 receive entry question(std txt rate)T&C\'s apply 08452810075over18\'sU dun say early hor... U c already say...Nah think goes usf, lives around thoughFreeMsg Hey darling 3 week\'s word back! I\'d like fun still? Tb ok! XxX std chgs send, å£1.50 rcvEven brother like speak me. treat like aids patent.As per request \'Melle Melle (Oru Minnaminunginte Nurungu Vettam)\' set callertune Callers. Press *9 copy friends CallertuneWINNER!! valued network customer selected receivea å£900 prize reward! claim call 09061701461. Claim code KL341. Valid 12 hours only.Had mobile 11 months more? U R entitled Update latest colour mobiles camera Free! Call Mobile Update Co FREE 08002986030I\'m gonna home soon want talk stuff anymore tonight, k? I\'ve cried enough today.SIX chances win CASH! 100 20,000 pounds txt> CSH

In [27]:
df['tokenized_message'] = df['message'].apply(text_process)

In [28]:
df.head()

,label,message,tokenized_message
0,ham,"Go until jurong point, crazy.. Available only ...",Go jurong point crazy Available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry 2 wkly comp win FA Cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,U dun say early hor U c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah dont think goes usf lives around though


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, TransformerMixin

In [40]:
vectorizer = TfidfVectorizer()
fit = vectorizer.fit_transform(list(df['tokenized_message']))

In [47]:
len(vectorizer.get_feature_names())

9376

In [49]:
sum(fit.toarray())

array([0.55869287, 0.26234975, 0.3529633 , ..., 0.31532655, 2.69219953,
       0.38499601])

In [52]:
fit.toarray().shape

(5572, 9376)

In [55]:
df.iloc[4065]['tokenized_message']

'Fyi Im gonna call sporadically starting like ltgt bc doin shit'

In [54]:
sum(fit.toarray()[4065])

3.162474967555524

##Traing model

In [69]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(fit.toarray(),df['label'],test_size=0.20,random_state=42)

In [70]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(4457, 9376)
(4457,)
(1115, 9376)
(1115,)


In [71]:
from sklearn.naive_bayes import GaussianNB
spam_detect_model = GaussianNB().fit(x_train,y_train)

In [72]:
train_preds = spam_detect_model.predict(x_train)
test_preds = spam_detect_model.predict(x_test)


In [75]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [76]:
# Confusion matrices for train and test 

print(confusion_matrix(y_train,train_preds))
print(confusion_matrix(y_test,test_preds))

[[3651  209]
 [   0  597]]
[[850 115]
 [ 16 134]]


In [81]:
print(accuracy_score(y_train,train_preds))
print(accuracy_score(y_test,test_preds))

0.9531074713933139
0.8825112107623319


In [84]:
print(classification_report(y_train,train_preds))

print(classification_report(y_test,test_preds))

              precision    recall  f1-score   support

         ham       1.00      0.95      0.97      3860
        spam       0.74      1.00      0.85       597

    accuracy                           0.95      4457
   macro avg       0.87      0.97      0.91      4457
weighted avg       0.97      0.95      0.96      4457

              precision    recall  f1-score   support

         ham       0.98      0.88      0.93       965
        spam       0.54      0.89      0.67       150

    accuracy                           0.88      1115
   macro avg       0.76      0.89      0.80      1115
weighted avg       0.92      0.88      0.89      1115

